| Step | What we do                                                | From                         |
| ---- | --------------------------------------------------------- | ---------------------------- |
| 1️⃣  | Input a **fundus image**                                  | IDRiD A/B                    |
| 2️⃣  | Predict **DR stage** using a **CNN classifier**           | Part B                       |
| 3️⃣  | Predict **lesions** using **UNet segmentation**           | Part A                       |
| 4️⃣  | Match the CNN stage prediction to the lesion evidence     | You! (via logic or Grad-CAM) |
| 5️⃣  | 💬 Output: "Model predicts Stage 2 due to Exudates + MAs" | YOUR GOAL!                   |


In [ ]:
pip install torch torchvision pandas

In [104]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
from torchvision import transforms
from glob import glob
from torch.utils.data import DataLoader
import torch


class IDRiDGradingDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)  # ✅ use read_csv
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        base_name = self.labels_df.iloc[idx, 0]  # e.g. IDRiD_361
        possible_files = glob(os.path.join(self.img_dir, base_name + ".*"))
        if not possible_files:
            raise FileNotFoundError(f"❌ Image not found for: {base_name}")
        img_path = possible_files[0]

        image = Image.open(img_path).convert("RGB")
        # label = int(self.labels_df.iloc[idx, 1])
        label = int(self.labels_df['Retinopathy grade'].iloc[idx])


        if self.transform:
            image = self.transform(image)

        return image, label


In [105]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [106]:
dataset = IDRiDGradingDataset(
    csv_file="C:/Users/VAIBHAV\Desktop/Programs/diabetic_retinopathy_detection/Datasets/Disease_Grading_CNN/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv",
    img_dir=r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Disease_Grading_CNN\1. Original Images\a. Training Set",
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [107]:
import torch.nn as nn
import torch.nn.functional as F

class DRStageClassifier(nn.Module):
    def __init__(self):
        super(DRStageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 54 * 54, 128)
        self.fc2 = nn.Linear(128, 5)  # 5 classes: 0 to 4

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (224 → 111)
        x = self.pool(F.relu(self.conv2(x)))  # (111 → 54)
        x = x.view(-1, 64 * 54 * 54)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [110]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [109]:
import os

img_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Disease_Grading_CNN\1. Original Images\a. Training Set"

files = os.listdir(img_dir)
extensions = set([os.path.splitext(f)[1] for f in files])
print("Extensions found:", extensions)

Extensions found: {'.jpg'}


In [111]:
model = DRStageClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"✅ Epoch {epoch+1}, Loss: {running_loss/len(dataloader):.4f}")

✅ Epoch 1, Loss: 1.9065
✅ Epoch 2, Loss: 1.3427
✅ Epoch 3, Loss: 1.2603
✅ Epoch 4, Loss: 1.1770
✅ Epoch 5, Loss: 1.1521
✅ Epoch 6, Loss: 1.0446
✅ Epoch 7, Loss: 0.9354
✅ Epoch 8, Loss: 0.8838
✅ Epoch 9, Loss: 0.8112
✅ Epoch 10, Loss: 0.7008


In [112]:
torch.save(model.state_dict(), "Models/cnn_dr_stage.pth")

In [113]:
model = DRStageClassifier().to(device)
model.load_state_dict(torch.load("Models/cnn_dr_stage.pth", map_location=device))
model.eval()

DRStageClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=186624, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

In [114]:
print(dataset.labels_df.columns)

Index(['Image name', 'Retinopathy grade', 'Risk of macular edema ',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')


In [115]:
correct = 0
total = 0

for images, labels in dataloader:
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        predicted = torch.argmax(outputs, 1)

    correct += (predicted == labels).sum().item()
    total += labels.size(0)

accuracy = (correct / total) * 100
print(f"✅ Model Accuracy on Dataset: {accuracy:.2f}%")


✅ Model Accuracy on Dataset: 80.87%


In [116]:
for i, (images, labels) in enumerate(dataloader):
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        predicted = torch.argmax(outputs, 1)

    for j in range(len(labels)):
        actual_label = labels[j].item()
        pred_label = predicted[j].item()
        result = "✅" if actual_label == pred_label else "❌"
        print(f"{result} Image {i*len(labels)+j+1}: Predicted {pred_label}, Actual {actual_label}")


✅ Image 1: Predicted 0, Actual 0
✅ Image 2: Predicted 1, Actual 1
✅ Image 3: Predicted 3, Actual 3
✅ Image 4: Predicted 4, Actual 4
❌ Image 5: Predicted 2, Actual 3
✅ Image 6: Predicted 2, Actual 2
✅ Image 7: Predicted 4, Actual 4
✅ Image 8: Predicted 0, Actual 0
✅ Image 9: Predicted 3, Actual 3
❌ Image 10: Predicted 0, Actual 3
❌ Image 11: Predicted 4, Actual 2
❌ Image 12: Predicted 0, Actual 4
❌ Image 13: Predicted 2, Actual 0
✅ Image 14: Predicted 0, Actual 0
❌ Image 15: Predicted 2, Actual 4
❌ Image 16: Predicted 2, Actual 0
❌ Image 17: Predicted 2, Actual 3
✅ Image 18: Predicted 2, Actual 2
✅ Image 19: Predicted 2, Actual 2
✅ Image 20: Predicted 2, Actual 2
✅ Image 21: Predicted 4, Actual 4
❌ Image 22: Predicted 1, Actual 2
✅ Image 23: Predicted 3, Actual 3
✅ Image 24: Predicted 0, Actual 0
✅ Image 25: Predicted 2, Actual 2
❌ Image 26: Predicted 2, Actual 3
✅ Image 27: Predicted 3, Actual 3
❌ Image 28: Predicted 2, Actual 3
✅ Image 29: Predicted 0, Actual 0
✅ Image 30: Predicted 3

🧠 Why Is This Happening?

DR stages can look similar, especially in early/mid stages

Your CNN has never “seen” features like hemorrhages, exudates — it’s just guessing patterns

Fundus images are complex; CNN alone has no semantic knowledge of retinal lesions

UNET MODEL TRAINING


In [117]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
            )

        self.encoder1 = conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.encoder2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.encoder3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        self.bottleneck = conv_block(256, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = conv_block(128, 64)

        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool1(e1))
        e3 = self.encoder3(self.pool2(e2))

        b = self.bottleneck(self.pool3(e3))

        d3 = self.up3(b)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.decoder3(d3)

        d2 = self.up2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.decoder2(d2)

        d1 = self.up1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.decoder1(d1)

        return torch.sigmoid(self.final(d1))


In [118]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [119]:
class EXSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_names = sorted(os.listdir(image_dir))
        self.mask_names = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_names[idx])

        image = Image.open(img_path).convert("RGB").resize((256, 256))
        mask = Image.open(mask_path).convert("L").resize((256, 256))

        image = transforms.ToTensor()(image)
        mask = transforms.ToTensor()(mask)

        return image, mask


1. Microaneurysms

In [120]:
image_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\a. Training Set"
mask_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\2. All Segmentation Groundtruths\a. Training Set\1. Microaneurysms"

dataset = EXSegmentationDataset(image_dir, mask_dir)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [121]:
model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"✅ Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(loader):.4f}")


✅ Epoch 1/10, Loss: 0.1812
✅ Epoch 2/10, Loss: 0.0336
✅ Epoch 3/10, Loss: 0.0342
✅ Epoch 4/10, Loss: 0.0338
✅ Epoch 5/10, Loss: 0.0344
✅ Epoch 6/10, Loss: 0.0340
✅ Epoch 7/10, Loss: 0.0348
✅ Epoch 8/10, Loss: 0.0344
✅ Epoch 9/10, Loss: 0.0347
✅ Epoch 10/10, Loss: 0.0335


In [122]:
torch.save(model.state_dict(), "Models/unet_ma.pth")
print("✅ MA model saved as unet_ma.pth")


✅ MA model saved as unet_ma.pth


In [123]:
from sklearn.metrics import accuracy_score
import numpy as np

unet_ma.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)
        outputs = unet_ma(images)
        preds = (outputs > 0.5).float()
        all_preds.append(preds.cpu().numpy().ravel())
        # Ensure masks are binary
        all_targets.append((masks > 0.5).cpu().numpy().ravel())

all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

# Calculate pixel-wise accuracy
pixel_accuracy = accuracy_score(all_targets, all_preds)
print(f"✅ Pixel-wise Accuracy of unet_ma: {pixel_accuracy*100:.2f}%")

✅ Pixel-wise Accuracy of unet_ma: 100.00%


3. Hard Exudates

In [124]:
image_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\a. Training Set"
mask_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\2. All Segmentation Groundtruths\a. Training Set\3. Hard Exudates"

dataset = EXSegmentationDataset(image_dir, mask_dir)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [125]:
model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"✅ Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(loader):.4f}")

✅ Epoch 1/10, Loss: 0.3455
✅ Epoch 2/10, Loss: 0.2433
✅ Epoch 3/10, Loss: 0.2454
✅ Epoch 4/10, Loss: 0.2478
✅ Epoch 5/10, Loss: 0.2536
✅ Epoch 6/10, Loss: 0.2427
✅ Epoch 7/10, Loss: 0.2543
✅ Epoch 8/10, Loss: 0.2483
✅ Epoch 9/10, Loss: 0.2410
✅ Epoch 10/10, Loss: 0.2495


In [126]:
torch.save(model.state_dict(), "Models/unet_ex.pth")
print("✅ EX model saved as unet_ex.pth")


✅ EX model saved as unet_ex.pth


2. Haemorrhages

In [127]:
class LesionSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, lesion_suffix="EX", transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.lesion_suffix = lesion_suffix

        all_imgs = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
        all_masks = sorted([f for f in os.listdir(mask_dir) if f.endswith(".tif")])

        # Extract basenames without suffix
        img_basenames = {os.path.splitext(f)[0] for f in all_imgs}
        mask_basenames = {os.path.splitext(f)[0].rsplit("_", 1)[0] for f in all_masks}

        # Match images and masks with same base
        common_basenames = sorted(list(img_basenames & mask_basenames))

        self.image_names = [f + ".jpg" for f in common_basenames]
        self.mask_names  = [f + f"_{self.lesion_suffix}.tif" for f in common_basenames]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        try:
            img_path = os.path.join(self.image_dir, self.image_names[idx])
            mask_path = os.path.join(self.mask_dir, self.mask_names[idx])

            image = Image.open(img_path).convert("RGB").resize((256, 256))
            mask = Image.open(mask_path).convert("L").resize((256, 256))

            image = transforms.ToTensor()(image)
            mask = transforms.ToTensor()(mask)

            return image, mask

        except Exception as e:
            print(f"❌ Error at index {idx}: {e}")
            return self.__getitem__((idx + 1) % len(self.image_names))


In [ ]:
# image_dir = r"C:\Users\HP\Desktop\gsfc\Sem7\MPII\implementation_trial\second\A. Segmentation\1. Original Images\a. Training Set"
# mask_dir = r"C:\Users\HP\Desktop\gsfc\Sem7\MPII\implementation_trial\second\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\2. Haemorrhages"

# dataset = LesionSegmentationDataset(image_dir, mask_dir, lesion_suffix="HE")

# print(f"🧾 Total matched image-mask pairs: {len(dataset)}")

# for i in range(min(len(dataset), 5)):
#     print(f"\n🔹 Image Name: {dataset.image_names[i]}")
#     print(f"🔹 Mask Name : {dataset.mask_names[i]}")


In [128]:
from torch.utils.data import DataLoader

image_dir = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\a. Training Set"
mask_dir  = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\2. All Segmentation Groundtruths\a. Training Set\2. Haemorrhages"

dataset = LesionSegmentationDataset(image_dir, mask_dir, lesion_suffix="HE")
loader = DataLoader(dataset, batch_size=4, shuffle=True)

print(f"🔥 Ready! Dataset length: {len(dataset)}")


🔥 Ready! Dataset length: 53


In [129]:
model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"✅ Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(loader):.4f}")


✅ Epoch 1/10, Loss: 0.3850
✅ Epoch 2/10, Loss: 0.3007
✅ Epoch 3/10, Loss: 0.2909
✅ Epoch 4/10, Loss: 0.3219
✅ Epoch 5/10, Loss: 0.2968
✅ Epoch 6/10, Loss: 0.3141
✅ Epoch 7/10, Loss: 0.3062
✅ Epoch 8/10, Loss: 0.2960
✅ Epoch 9/10, Loss: 0.2920
✅ Epoch 10/10, Loss: 0.3222


In [130]:
torch.save(model.state_dict(), "Models/unet_he.pth")
print("✅ The model saved as unet_he.pth")


✅ The model saved as unet_he.pth


UNET Models integrating with CNN Model

In [131]:
unet_ex = UNet().to(device)   # HARD Exudates
unet_ma = UNet().to(device)   # Microaneurysms
unet_he = UNet().to(device)   # Hemorrhages

# Load trained weights
# unet_ex.load_state_dict(torch.load("unet_ex.pth"))
# unet_ma.load_state_dict(torch.load("unet_ma.pth"))
# unet_he.load_state_dict(torch.load("unet_he.pth"))

unet_ex.load_state_dict(torch.load(r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Models\unet_ex.pth"))
unet_ex.eval()
unet_ma.load_state_dict(torch.load(r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Models\unet_ma.pth"))
unet_ma.eval()
unet_he.load_state_dict(torch.load(r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Models\unet_he.pth"))
unet_he.eval()


UNet(
  (encoder1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bottleneck): Sequen

In [132]:
print("Model eval mode:", unet_ex.training)
for name, param in unet_ex.named_parameters():
    print(f"{name}: mean={param.data.mean().item():.4f}, std={param.data.std().item():.4f}")
    break  # Only print the first parameter


print(sum(p.numel() for p in model.parameters()))
print(sum(p.nonzero().size(0) for p in model.parameters()))



Model eval mode: False
encoder1.0.weight: mean=0.0037, std=0.1105
7697345
7697345


In [161]:
import torch
import numpy as np
from torchvision.transforms import ToTensor
from PIL import Image

def predict_mask(unet_model, image_path, device=device, threshold=0.1, resize=(256,256)):
    # Load and preprocess image
    img = Image.open(image_path).convert("RGB")
    orig_size = img.size  # (W,H)
    img_resized = img.resize(resize)
    tensor = ToTensor()(img_resized).unsqueeze(0).to(device)

    # Predict
    unet_model.eval()
    with torch.no_grad():
        pred = unet_model(tensor)

        # If binary model -> apply sigmoid
        if pred.shape[1] == 1:
            prob = torch.sigmoid(pred)
            mask = (prob > threshold).float()
            mask = mask.squeeze().cpu().numpy()
        else:
            # Multi-class segmentation -> take argmax
            prob = torch.softmax(pred, dim=1)
            mask = torch.argmax(prob, dim=1).squeeze().cpu().numpy()

    # Resize back to original image size
    mask_img = Image.fromarray((mask*255).astype(np.uint8))
    mask_img = mask_img.resize(orig_size, resample=Image.NEAREST)
    mask = np.array(mask_img) // 255  # binary 0/1

    return mask


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# def show_mask_overlay(image, mask, title="Mask Overlay", color="red", alpha=0.4):
#     """
#     image: PIL Image (original)
#     mask: numpy array [H, W] (binary mask)
#     color: overlay color (default red)
#     alpha: transparency (0-1)
#     """
#     image = np.array(image)

#     # Create colored mask
#     overlay = np.zeros_like(image)
#     if color == "red":
#         overlay[..., 0] = 255
#     elif color == "green":
#         overlay[..., 1] = 255
#     elif color == "blue":
#         overlay[..., 2] = 255

#     # Apply mask
#     mask_bool = mask.astype(bool)
#     blended = image.copy()
#     blended[mask_bool] = (alpha * overlay[mask_bool] + (1 - alpha) * blended[mask_bool]).astype(np.uint8)

#     plt.figure(figsize=(6,6))
#     plt.imshow(blended)
#     plt.axis("off")
#     plt.title(title)
#     plt.show()


In [ ]:
# img_path = r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\b. Testing Set\IDRiD_66.jpg"

# # Predict with UNet (Exudates model for example)
# image, ex_mask = predict_mask(unet_ex, img_path, threshold=0.4)

# # Show overlay
# show_mask_overlay(image, ex_mask, title="Exudates Prediction", color="red")


In [ ]:
# image, ex_mask = predict_mask(unet_ex, img_path, threshold=0.4)
# _, ma_mask = predict_mask(unet_ma, img_path, threshold=0.4)
# _, he_mask = predict_mask(unet_he, img_path, threshold=0.4)

# # Show overlays
# show_mask_overlay(image, ex_mask, title="Exudates (Red)", color="red")
# show_mask_overlay(image, ma_mask, title="Microaneurysms (Green)", color="green")
# show_mask_overlay(image, he_mask, title="Hemorrhages (Blue)", color="blue")


In [165]:
def get_detected_lesions(image_path):
    lesion_detected = []
    
    unet_ex.eval()
    unet_ma.eval()
    unet_he.eval()
    
    ex_mask = predict_mask(unet_ex, image_path)
    ma_mask = predict_mask(unet_ma, image_path)
    he_mask = predict_mask(unet_he, image_path)
    
    print("ex_mask : ",ex_mask)
    print("ma_mask : ",ma_mask.sum())
    print("he_mask : ", he_mask.sum())

    if ex_mask.sum() > 1000:
        lesion_detected.append("Exudates")
    if ma_mask.sum() > 1000:
        lesion_detected.append("Microaneurysms")
    if he_mask.sum() > 1000:
        lesion_detected.append("Hemorrhages")

    return lesion_detected


In [139]:
stage_labels = {
    0: "No DR",
    1: "Mild DR",
    2: "Moderate DR",
    3: "Severe DR",
    4: "Proliferative DR"
}

def explain_prediction(image_path):
    pred_stage = cnn_predict_stage(image_path)
    lesions = get_detected_lesions(image_path)

    print("📸 Image:", image_path.split("/")[-1])
    print(f"🧠 CNN Predicted Stage: {pred_stage} ({stage_labels[pred_stage]})")

    if lesions:
        print(f"🔬 Lesions Detected: {', '.join(lesions)}")
        print(f"✅ Explanation: DR stage {pred_stage} likely due to presence of {', '.join(lesions)}")
    else:
        print("⚠️ No significant lesions detected in this image.")


In [140]:
def cnn_predict_stage(img_path):
    cnn_model = DRStageClassifier().to(device)
    cnn_model.load_state_dict(torch.load("Models/cnn_dr_stage.pth", map_location=device))
    cnn_model.eval()
    img = Image.open(img_path).convert("RGB").resize((224, 224))
    tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = cnn_model(tensor)
        print("🧪 CNN Output Shape:", output.shape)

        pred = torch.argmax(output, dim=1).item()

    return pred

In [168]:
explain_prediction(r"C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\b. Testing Set\IDRiD_66.jpg")


🧪 CNN Output Shape: torch.Size([1, 5])
ex_mask :  [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
ma_mask :  12212224
he_mask :  12212224
📸 Image: C:\Users\VAIBHAV\Desktop\Programs\diabetic_retinopathy_detection\Datasets\Segmentation_for_Unet\1. Original Images\b. Testing Set\IDRiD_66.jpg
🧠 CNN Predicted Stage: 3 (Severe DR)
🔬 Lesions Detected: Exudates, Microaneurysms, Hemorrhages
✅ Explanation: DR stage 3 likely due to presence of Exudates, Microaneurysms, Hemorrhages


In [ ]:
explain_prediction(r"C:\Users\HP\Desktop\gsfc\Sem7\MPII\implementation_trial\second\A. Segmentation\1. Original Images\a. Training Set\IDRiD_01.jpg")


So when it says:

⚠️ No significant lesions detected

It means:
❗Your UNet model didn't detect any lesion regions for that image — even though there’s a mask for it in groundtruth.

This could be due to:

UNet not learning properly

Threshold too high

Prediction too soft (low confidence)

Or simply: image size mismatch or wrong preprocessing